In [1]:
import numpy as np
import pandas as pd
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial







In [2]:
data = pd.read_csv(r"C:\Users\Lokesh\Downloads\kmeans_data\data.csv")
labels = pd.read_csv(r"C:\Users\Lokesh\Downloads\kmeans_data\label.csv",names=['label'],header=None)

In [3]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [5]:
#Split data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [6]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
    
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           
        return centroid_value_dict,centroid_dict
    

In [7]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [8]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [9]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [25]:
model_x = KMeans()
centroids1,clusters1 = model_x.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  26835.021276595744
Tolerance for the Iteration  1 :  8089.031606537157
Tolerance for the Iteration  2 :  4577.67268844628
Tolerance for the Iteration  3 :  3244.790173517833
Tolerance for the Iteration  4 :  2543.2231515370804
Tolerance for the Iteration  5 :  2016.733187138835
Tolerance for the Iteration  6 :  1743.6473490541593
Tolerance for the Iteration  7 :  1214.7696585903084
Tolerance for the Iteration  8 :  1171.4062861271677
Tolerance for the Iteration  9 :  988.257009716988
Tolerance for the Iteration  10 :  951.721840057171
Tolerance for the Iteration  11 :  996.118424091266
Tolerance for the Iteration  12 :  852.8440161988551
Tolerance for the Iteration  13 :  851.1574336969385
Tolerance for the Iteration  14 :  880.4474538312295
Tolerance for the Iteration  15 :  891.8852054371332
Tolerance for the Iteration  16 :  1026.1602077247644
Tolerance for the Iteration  17 :  1157.9671049257527
Tolerance for the Iteration  18 :  1230.21227519196
T

In [11]:
Euclidean_SSE = model_x.calculate_SSE(centroids1,clusters1,data)

In [12]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25321672765.52289


In [13]:
cluster_labels_a = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels_a

array([5, 3, 0, 4, 6, 0, 7, 1, 2, 9])

In [14]:
Euclidean_Acc = accuracy(centroids1, cluster_labels_a,test_data,test_labels)
Euclidean_Acc

0.08875

In [15]:
model_y = KMeans()
centroids2,clusters2 = model_y.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = model_y.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  37428.56123822341
Tolerance for the Iteration  1 :  3177.602712860804
Tolerance for the Iteration  2 :  1400.868024017288
Tolerance for the Iteration  3 :  2126.5286869657316
Tolerance for the Iteration  4 :  3095.095201272705
Tolerance for the Iteration  5 :  1016.1113896881915
Tolerance for the Iteration  6 :  780.9275363777988
Tolerance for the Iteration  7 :  1875.8730362192846
Tolerance for the Iteration  8 :  1038.265967811356
Tolerance for the Iteration  9 :  2114.52637337134
Tolerance for the Iteration  10 :  1308.3374630613744
Tolerance for the Iteration  11 :  1269.4103742052457
Tolerance for the Iteration  12 :  0.0


In [16]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [17]:
cluster_labels_b = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels_b

array([1, 1, 6, 3, 0, 1, 7, 6, 4, 7])

In [18]:
Jaccard_Acc = accuracy(centroids2, cluster_labels_b,test_data,test_labels)
Jaccard_Acc

0.1075

In [26]:
model_z = KMeans()
centroids3,clusters3 = model_z.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  26372.742817098806
Tolerance for the Iteration  1 :  6573.116861296968
Tolerance for the Iteration  2 :  3474.2398364916194
Tolerance for the Iteration  3 :  2516.180864197531
Tolerance for the Iteration  4 :  2266.456030389363
Tolerance for the Iteration  5 :  2253.115678730883
Tolerance for the Iteration  6 :  3989.8399908172632
Tolerance for the Iteration  7 :  3685.77902237927
Tolerance for the Iteration  8 :  2166.5770375965553
Tolerance for the Iteration  9 :  2151.413541352628
Tolerance for the Iteration  10 :  2090.9093569750053
Tolerance for the Iteration  11 :  1274.9357447792377
Tolerance for the Iteration  12 :  1202.500276054953
Tolerance for the Iteration  13 :  880.1679785838386
Tolerance for the Iteration  14 :  694.4094304388423
Tolerance for the Iteration  15 :  478.4292209151335
Tolerance for the Iteration  16 :  381.3271785886092
Tolerance for the Iteration  17 :  263.7121729270705
Tolerance for the Iteration  18 :  194.373024766926

In [20]:
Cosine_SSE = model_z.calculate_SSE(centroids3,clusters3,data)

In [21]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25321672765.52289
Jacard SSE: 34361687572.938736
Cosine SSE : 25553361774.674137


In [22]:
cluster_labels_c = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels_c

array([9, 3, 0, 2, 1, 7, 0, 3, 8, 4])

In [23]:
Cosine_Acc = accuracy(centroids3, cluster_labels_c,test_data,test_labels)
Euclidean_Acc
Jaccard_Acc
Cosine_Acc

0.07625

In [24]:
print("Euclidean accuracy:",Euclidean_Acc)
print("Jacard accuracy:",Jaccard_Acc)
print("Cosine accuracy :",Cosine_Acc)

Euclidean accuracy: 0.08875
Jacard accuracy: 0.1075
Cosine accuracy : 0.07625
